<h1 style='background:lightblue; border:0; color:black'><center>Health Insurance Cross Sell Prediction.</center></h1>



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install seaborn --upgrade 

<center><img src="https://www.royalsundaram.in/html/files/tips/vehicle-insurance-types-and-how-to-claim-it.png"></center>

<h1 style='background:lightblue; border:0; color:black'><center>Understanding the problem.</center></h1>


Our goal is to find a model to an insurance company that has provided Health Insurance to its customers. now they want build a model to predict whether the policyholders (customers) from past year will also be interested in Vehicle Insurance provided by the company. 

An insurance policy is an arrangement by which a company undertakes to provide a guarantee of compensation for specified loss, damage, illness, or death in return for the payment of a specified premium. A premium is a sum of money that the customer needs to pay regularly to an insurance company for this guarantee.

There is vehicle insurance where every year customer needs to pay a premium of certain amount to insurance provider company so that in case of unfortunate accident by the vehicle, the insurance provider company will provide a compensation (called ‘sum assured’) to the customer.

Building a model to predict whether a customer would be interested in Vehicle Insurance is extremely helpful for the company because it can then accordingly plan its communication strategy to reach out to those customers and optimise its business model and revenue.

Now, in order to predict, whether the customer would be interested in Vehicle insurance, you have information about demographics (gender, age, region code type), Vehicles (Vehicle Age, Damage), Policy (Premium, sourcing channel) etc.

<a id="1"></a>
<h2 style='background:lightblue; border:0; color:black'><center>1. Exploratory Data Analysis<center><h2>


### Load the data

In [ ]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import optuna
from optuna.samplers import TPESampler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier


In [ ]:
train = pd.read_csv("../input/health-insurance-cross-sell-prediction/train.csv")
sample_submission = pd.read_csv("../input/health-insurance-cross-sell-prediction/sample_submission.csv")

In [ ]:
train

### Looking for missing values

In [ ]:
train.isnull().sum()

 ### Changing labels




Gender-> Male=0, Female=1

Vehicle_Age-> <1 Year=0, 1-2 Year=1,>2 Years=2

Vehicle_Damage->0=No Yes=1

In [ ]:
train.loc[(train['Gender'] =="Male"), 'Gender'] = 0.0
train.loc[(train['Gender'] =="Female"), 'Gender'] = 1.0
train.loc[(train['Vehicle_Age'] =="< 1 Year"), 'Vehicle_Age'] = 0.0
train.loc[(train['Vehicle_Age'] =="1-2 Year"), 'Vehicle_Age'] = 1.0
train.loc[(train['Vehicle_Age'] =="> 2 Years"), 'Vehicle_Age'] = 2.0
train.loc[(train['Vehicle_Damage'] =="No"), 'Vehicle_Damage'] = 0.0
train.loc[(train['Vehicle_Damage'] =="Yes"), 'Vehicle_Damage'] = 1.0

train=train.drop(['id'],axis=1)
train["Vehicle_Damage"] = pd.to_numeric(train["Vehicle_Damage"])
train["Vehicle_Age"] = pd.to_numeric(train["Vehicle_Age"])
train["Gender"] = pd.to_numeric(train["Gender"])

In [ ]:
train.head()

### Percentage of customers insured

In [ ]:
total = float(len(train)) 
ax = sns.countplot(x="Response", data=train, palette="tab10") 
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format(height/total),
            ha="center") 

### Customers by gender

In [ ]:
total = float(len(train)) 
ax = sns.countplot(x="Gender", data=train,hue="Response", palette="tab10") 
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format(height/total),
            ha="center") 

### Customers by Driving License

In [ ]:
total = float(len(train)) 
ax = sns.countplot(x="Driving_License", data=train,hue="Response", palette="tab10") 
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format(height/total),
            ha="center") 

In [ ]:
train['Driving_License'].value_counts()


### Customers by previously insured

In [ ]:
total = float(len(train)) 
ax = sns.countplot(x="Previously_Insured", data=train,hue="Response", palette="tab10") 
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format(height/total),
            ha="center") 

### Customers by vehicle age

In [ ]:
total = float(len(train)) 
ax = sns.countplot(x="Vehicle_Age", data=train,hue="Response", palette="tab10") 
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format(height/total),
            ha="center") 

### Customers by vehicle damage

In [ ]:
total = float(len(train)) 
ax = sns.countplot(x="Vehicle_Damage", data=train,hue="Response", palette="tab10") 
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format(height/total),
            ha="center") 

### Continuous variables


In [ ]:
ContV= pd.concat([train['Age'], train['Annual_Premium'],train['Policy_Sales_Channel'],train['Vintage'],train['Region_Code'],train['Response']], axis=1)

In [ ]:
sns.pairplot(ContV.sample(10000, random_state=0), hue="Response", height=2.5)


### Correlation

In [ ]:
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(train.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

<a id="2"></a>
<h2 style='background:lightblue; border:0; color:black'><center>2. Modelling<center><h2>

### Preparing the training a test set

In [ ]:
X=train.drop(['Response'],axis=1)
y=train['Response']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12345)

### The logistic regression

In [ ]:
model_LR = LogisticRegression(random_state=12345)
model_LR.fit(X_train, y_train)
pred_LR = model_LR.predict(X_test)


In [ ]:
print(accuracy_score(y_test, pred_LR))
print(f1_score(y_test, pred_LR))

In [ ]:
LR_score=accuracy_score(y_test, pred_LR)

In [ ]:
# calculate the fpr and tpr for all thresholds of the classification
probs = model_LR.predict_proba(X_test)
predp_LR = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, predp_LR)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
roc_auc_score(y_test,predp_LR)

### Random forest

Search the best parameters

In [ ]:
sampler = TPESampler(seed=0)
def create_modelRF(trial):
    max_depth = trial.suggest_int("max_depth", 2, 20)
    n_estimators = trial.suggest_int("n_estimators", 2, 300)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    model = RandomForestClassifier(min_samples_leaf=min_samples_leaf,
                                   n_estimators=n_estimators, max_depth=max_depth, random_state=0)
    return model

def objective(trial):
    model = create_modelRF(trial)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    predp_LR = probs[:,1]
    score = roc_auc_score(y_test, predp_LR)
    return score

In [ ]:
'''study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=100)
params_rf = study.best_params'''



In [ ]:
params_rf={'max_depth': 15,
 'n_estimators': 208,
 'min_samples_leaf': 10,
 'min_samples_split': 9}

In [ ]:
model_rf = RandomForestClassifier(**params_rf)
model_rf.fit(X_train,y_train)
pred_rf = model_rf.predict(X_test)
print('Optimized RF accuracy: ', accuracy_score(y_test, pred_rf))
print('Optimized RF f1-score:', f1_score(y_test, pred_rf))

In [ ]:
rf_score=accuracy_score(y_test, pred_rf)

In [ ]:
# calculate the fpr and tpr for all thresholds of the classification
probs = model_rf.predict_proba(X_test)
predp_rf = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, predp_rf)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
roc_auc_score(y_test,predp_rf)

### Adaboost

In [ ]:
sampler = TPESampler(seed=0)
def create_modelada(trial):
    n_estimators = trial.suggest_int("n_estimators", 2, 300)
    model = AdaBoostClassifier(n_estimators=n_estimators, random_state=0)
    return model

def objective(trial):
    model = create_modelada(trial)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    predp_ada = probs[:,1]
    score = roc_auc_score(y_test, predp_ada)
    return score

In [ ]:
'''study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=100)
params_ada = study.best_params'''


In [ ]:
params_ada={'n_estimators': 210}


In [ ]:
model_ada = AdaBoostClassifier(**params_ada)
model_ada.fit(X_train, y_train)
pred_ada=model_ada.predict(X_test)

In [ ]:
print('Optimized RF accuracy: ', accuracy_score(y_test, pred_ada))
print('Optimized RF f1-score:', f1_score(y_test, pred_ada))

In [ ]:
ada_score= accuracy_score(y_test, pred_ada)

In [ ]:
# calculate the fpr and tpr for all thresholds of the classification
probs = model_ada.predict_proba(X_test)
predp_ada = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, predp_ada)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
roc_auc_score(y_test,predp_ada)

###  LightGBM 

In [ ]:
def create_modelLGBM(trial):
    num_leaves = trial.suggest_int("num_leaves", 2, 31)
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 8)
    min_child_samples = trial.suggest_int('min_child_samples', 100, 1200)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0001, 0.99)
    model = LGBMClassifier(
        num_leaves=num_leaves,
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        min_child_samples=min_child_samples, 
        learning_rate=learning_rate,
        random_state=0
    )
    return model

def objective(trial):
    model = create_modelLGBM(trial)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    predp_LGBM = probs[:,1]
    score = roc_auc_score(y_test, predp_LGBM)
    return score

In [ ]:
'''study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=500)
params_LGBM = study.best_params'''


In [ ]:
params_LGBM={'num_leaves': 12,
 'n_estimators': 279,
 'max_depth': 7,
 'min_child_samples': 411,
 'learning_rate': 0.0966077053983753}

In [ ]:
model_LGBM = LGBMClassifier(**params_LGBM)
model_LGBM.fit(X_train, y_train)

pred_LGBM = model_LGBM.predict(X_test)
print('Simple LGBM accuracy: ', accuracy_score(y_test, pred_LGBM))
print('Simple LGBM Regression f1_score: ', f1_score(y_test, pred_LGBM))

In [ ]:
LGBM_score=accuracy_score(y_test, pred_LGBM)

In [ ]:
# calculate the fpr and tpr for all thresholds of the classification
probs = model_LGBM.predict_proba(X_test)
predp_LGBM = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, predp_LGBM)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
roc_auc_score(y_test,predp_LGBM)

### XGBoost

In [ ]:
sampler = TPESampler(seed=0)

def create_modelXGB(trial):
    max_depth = trial.suggest_int("max_depth", 2, 20)
    n_estimators = trial.suggest_int("n_estimators", 1, 400)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0000001, 1)
    gamma = trial.suggest_uniform('gamma', 0.0000001, 1)
    scale_pos_weight = trial.suggest_int("scale_pos_weight", 1, 20)
    model = XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators, 
                          max_depth=max_depth, gamma=gamma, scale_pos_weight=scale_pos_weight, random_state=0)
    return model

def objective(trial):
    model = create_modelXGB(trial)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    predp_XGM = probs[:,1]
    score = roc_auc_score(y_test, predp_XGM)
    return score


In [ ]:
'''study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=500)
params_xgb = study.best_params'''


In [ ]:
params_xgb={'max_depth': 3, 'n_estimators': 317, 'learning_rate': 0.14077264483362187, 'gamma': 0.15676425072562297, 'scale_pos_weight': 3}

In [ ]:
model_xgb = XGBClassifier(**params_xgb)
model_xgb.fit(X_train, y_train)
pred_xgb = model_xgb.predict(X_test)
print('Optimized XGBClassifier accuracy: ', accuracy_score(y_test, pred_xgb))
print('Optimized XGBClassifier f1-score', f1_score(y_test, pred_xgb))

In [ ]:
xgb_score=accuracy_score(y_test, pred_xgb)

In [ ]:
# calculate the fpr and tpr for all thresholds of the classification
probs = model_xgb.predict_proba(X_test)
predp_xgb = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, predp_xgb)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
roc_auc_score(y_test,predp_xgb)

### Essembling by score

In [ ]:
total_score=xgb_score+LGBM_score+ada_score+LR_score+rf_score

In [ ]:
g_xgb=(1-xgb_score/total_score)
g_LGB=(1-LGBM_score/total_score)
g_ada=(1-ada_score/total_score)
g_LR=(1-LR_score/total_score)
g_rf=(1-rf_score/total_score)

In [ ]:
f_pred=1/4*(g_xgb*pred_xgb+g_rf*pred_rf+g_LR*pred_LR+g_LGB*pred_LGBM+g_ada*pred_ada)

In [ ]:
f_predp=1/4*(g_xgb*predp_xgb+g_rf*predp_rf+g_LR*predp_LR+g_LGB*predp_LGBM+g_ada*predp_ada)

In [ ]:
accuracy_score(y_test, np.round(f_predp))

In [ ]:

fpr, tpr, threshold = metrics.roc_curve(y_test, f_predp)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
roc_auc_score(y_test,f_predp)